In [1]:
# Here we experiment on how to select the best subset of 
# dedispersed time series to search based on FFA search parameters.
import os
import glob
import json

%matplotlib qt5

import numpy as np
import matplotlib.pyplot as plt

from riptide import TimeSeries, Metadata
from riptide.reading import PrestoInf

In [101]:
# 1. Get dm_max
# 2. Get the list of ideal dm trials
# 3. Get the list of available time series
# 4. Get the list of available time series closest to each of the dm trials
def dispersion_smearing_time(dm, freq, bw):
    kdm = 4.148808e3
    return 2 * kdm * dm * freq**-3 * bw

def dispersion_measure(delay, fmin, fmax):
    """ Dispersion measure as function of dispersion delay between two given frequencies """
    kdm = 4.148808e3
    return delay / (kdm * abs(fmin**-2 - fmax**-2))

def get_dm_max(skycoord, dmsinb_max=40.0, dm_max=1000.0):
    """ Get an estimate of the maximum Galactic DM at given coordinates. """
    glat_radians = skycoord.galactic.b.rad
    eps = 1e-6
    return min(dm_max, dmsinb_max / np.sin(abs(glat_radians) + eps))

def get_optimal_dm_trials(dm_min=0.0, dm_max=1000.0, tsamp=64e-6, wmin=2048e-6, fbot=1181.0, ftop=1581.0, nchans=1024):
    fcenter = (fbot + ftop) / 2.0
    cbw = abs(ftop - fbot) / nchans
    
    dm = dm_min 
    trials = []
    while dm < dm_max:
        trials.append(dm)
        
        tdisp = dispersion_smearing_time(dm, fcenter, cbw)
        wsmear = (tsamp**2 + tdisp**2) ** 0.5
        
        # NOTE: This is basically saying:
        # - We cannot resolve any event with an effective (post-smearing) width < Wmin
        # - Below a certain DM value, the total smearing is lower than Wmin. Wmin determines the (fixed) DM step.
        # - Above a certain DM value, the total smearing exceeds Wmin. It is now the
        #   smearing that drives the DM trial spacing, which starts increasing as a function of DM.
        half_step = dispersion_measure(max(wmin, wsmear), fbot, ftop)
        dm = dm + 2 * half_step
        
    return np.asarray(trials)

#############################################################################################
    
def load_time_series_metadata(directory, pattern='*.inf', fmt='presto'):
    """ Returns a list of Metadata objects sorted by increasing DM. """
    fnames = glob.glob(os.path.join(directory, pattern))
    fmt = fmt.lower()
    if fmt == 'presto':
        loader = Metadata.from_presto_inf
    elif fmt == 'sigproc':
        loader = Metadata.from_sigproc
    else:
        raise ValueError("Invalid input data format specifier '{:s}'".format(fmt))
    
    mdlist = []
    for fname in fnames:
        md = loader(fname)
        # NOTE: add a fname key with the full file path
        md['fname'] = fname
        md['marked'] = False
        mdlist.append(md)
    return sorted(mdlist, key=lambda md: md['dm'])
    

def time_series_iterator(directory, pattern='*.inf', fmt='presto', wmin=1024e-6, fbot=1181.0, ftop=1581.0, nchans=1024, dm_min=0.0, dmsinb_max=40.0, dm_max=400.0):
    mdlist = load_time_series_metadata(directory, pattern=pattern, fmt=fmt)
    mdlist = sorted(mdlist, key=lambda md: md['dm'])
    
    # Get minimum sampling time among target time series
    tsamp = min(md['tsamp'] for md in mdlist)
    print("tsamp = {:.6e}".format(tsamp))
    
    # Get target coordinates
    skycoord = mdlist[0]['skycoord']
    print(skycoord)
    
    # Get DM limits
    dm_max = get_dm_max(skycoord, dmsinb_max=dmsinb_max, dm_max=dm_max)
    print("dm_max = {:.3f}".format(dm_max))
    
    # Get optimal DM trials
    optimal_dm_trials = get_optimal_dm_trials(
        dm_min=dm_min, dm_max=dm_max, tsamp=tsamp, wmin=wmin, fbot=fbot, ftop=ftop, nchans=nchans
        )
    print("optimal number of DM trials = {:d}".format(optimal_dm_trials.size))
    
    # Mark time series to search
    available_dm_trials = np.asarray([md['dm'] for md in mdlist])
    marked_indices = np.digitize(optimal_dm_trials, available_dm_trials) - 1
    marked_indices = np.unique(np.maximum(0, marked_indices))
    
    # Get loader function
    fmt = fmt.lower()
    if fmt == 'presto':
        loader = TimeSeries.from_presto_inf
    elif fmt == 'sigproc':
        loader = TimeSeries.from_sigproc
    else:
        raise ValueError("Invalid input data format specifier '{:s}'".format(fmt))
    
    for ix in marked_indices:
        fname = mdlist[ix]['fname']
        yield loader(fname)
    


In [105]:
directory = '/home/vince/work/pulsars/presto_time_series/J1855+0307/'
pattern = '*.inf'
load_time_series_metadata(directory, pattern=pattern, fmt='presto')

for x in time_series_iterator(directory, pattern=pattern, dm_min=360.0, dm_max=440.0, wmin=10e-3):
    print(x.metadata['dm'])


tsamp = 6.400000e-05
<SkyCoord (ICRS): (ra, dec) in deg
    ( 283.7685,  3.04411111)>
dm_max = 440.000
optimal number of DM trials = 6
360.0
372.0
388.0
404.0
420.0
436.0


In [9]:
dm_trials = get_optimal_dm_trials(dm_min=0.0, dm_max=500.0, tsamp=491.52e-6, wmin=2e-3, fbot=103.61, ftop=135.25, nchans=2592)

In [135]:
plt.figure()
plt.plot(dm_trials[:-1], np.diff(dm_trials))
plt.grid()
plt.show()

In [10]:
dm_trials.size

4948

In [130]:
dispersion_smearing_time(dm_trials[1]/2, 119.4, 0.01221)

7.677063139508884e-07

In [131]:
(2e-3 **2 + 0.49152e-3 ** 2) **0.5

0.002059512541938019

NameError: name 'directory' is not defined

In [52]:
load_time_series_metadata(directory)[0]

Metadata
{'analyst': 'Scott Ransom',
 'bandwidth': 400.0,
 'barycentered': True,
 'basename': 'J1855+0307_DM360.00',
 'bdiam': 981.0,
 'breaks': False,
 'cbw': 0.390625,
 'decj': '03:02:38.8000',
 'dm': 360.0,
 'fbot': 1182.1953125,
 'instrument': '????',
 'mjd': 56937.40513649563,
 'nchan': 1024,
 'nsamp': 8754411,
 'observer': 'Unknown',
 'obstype': 'Radio',
 'raj': '18:55:04.4400',
 'skycoord': <SkyCoord (ICRS): (ra, dec) in deg
    ( 283.7685,  3.04411111)>,
 'source': 'G036.5+00.6_s',
 'source_name': 'G036.5+00.6_s',
 'telescope': 'Parkes',
 'tobs': None,
 'tsamp': 6.4e-05}

In [87]:
np.digitize([1], [0,1,2,3,4]) - 1

array([1])